<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>Автор материала: Екатерина Ширяева slackname: Katya.Shiryaeva

#  Есть такая либа H20

Согласно исследованиям [Gartner в феврале 2018](https://www.gartner.com/doc/reprints?id=1-4RQ3VEZ&ct=180223&st=sb), H2O занимает уверенное место в лидерах рынка среди DataScience и Machine Learning платформ.
Gartner считают H2O.ai технологическим лидером, эта платформа мспользуется более чем 100000 data scientistами и удоволетверенность клиентами самая высокая (поддержка, обучение и продажи).  
В этом обзоре я хочу показать отличия от реализаций алгоритмов в обычном sklearn

<img src="https://www.gartner.com/resources/326400/326456/326456_0001.png;wa8addaec1755a1f0c?reprintKey=1-4RQ3VEZ" alt="Drawing" style="width: 500px;"/>

### Подготовка данных

Для примера можно взять датасет из [1ой лекции: отток клиентов телекома](https://habrahabr.ru/company/ods/blog/322626/)  

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('data/telecom_churn.csv')

In [ ]:
df.info()

Описание всех признаков можно посмотреть в [1ой лекции](https://habrahabr.ru/company/ods/blog/322626/)  
В качестве предварительной обработки заменю все значения churn на 0/1,   
для International plan и Voice mail plan сделаю замену yes / no на 0 / 1  
а категориальную переменную State пока удалю из обработки (данных недостаточно, чтобы сделать OHE, и укрупнение категорий не стоит в целях этого тьюториала)

In [ ]:
d = {'No' : 0, 'Yes' : 1}
df['Churn'] = df['Churn'].apply(lambda x : int(x))
df['International plan'] = df['International plan'].map(d)
df['Voice mail plan'] = df['Voice mail plan'].map(d)
df.drop('State', axis=1, inplace=True)

df.head() 

Разделим на тест и обучающую выборки

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.3, random_state=42)

Для работы h2o необходимо установить эту библиотеку и запустить

In [ ]:
!pip install h2o

In [ ]:
import os

import h2o

h2o.init(nthreads=-1, max_mem_size=8)
# nthreads - количество ядер процессора для вычислений
# max_mem_size - максимальный размер оперативной памяти 

Все данные надо перевести в специальную структуру h2o : H2OFrame  
h2o [поддерживает](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/getting-data-into-h2o.html) большое количество источников, однако у меня не получилось перекодировать csr-матрицу из задания про Элис :)  очень долго висел :)

In [ ]:
training = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
validation = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

Посмотрим на структуру (это обязательно надо делать, т.к. не всегда корректно происходит переход форматов) <br>
Здесь будут описаны основные параметры каждой переменной (тип, максимум, минимум, среднее, стандартное отклонение, количество нулевых и пропущенных значений и первые 10 наблюдений)

In [ ]:
training.describe()

Зависимая переменная для бинарной классификации должна быть не количественной переменной, а категориальной, преобразуем с помощью метода asfactor()

In [ ]:
training['Churn'] = training['Churn'].asfactor()
validation['Churn'] = validation['Churn'].asfactor()

training['International plan'] = training['International plan'].asfactor()
training['Voice mail plan'] = training['Voice mail plan'].asfactor()

validation['International plan'] = validation['International plan'].asfactor()
validation['Voice mail plan'] = validation['Voice mail plan'].asfactor()

### Random Forest

#### sklearn RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
forest = RandomForestClassifier(n_estimators=800, random_state=152, n_jobs=-1)
forest.fit(X_train, y_train)
print('AUC для sklearn RandomForestClassifier: {:.4f}'.format(roc_auc_score(y_test, forest.predict_proba(X_test)[:, 1])))

#### H2ORandomForestEstimator

In [ ]:
from h2o.estimators import H2ORandomForestEstimator

необходимо задать список зависимых переменных и предикторов (это будут X и y в коде)

In [ ]:
X = training.columns
X.remove('Churn')
y = 'Churn'

rf1 = H2ORandomForestEstimator(model_id='tutorial1', ntrees=800, seed=152)
rf1.train(X, y, training_frame=training, validation_frame=validation)
rf1

Получился большой вывод информации о модели.  
Сразу следует обратить внимание на AUC при тех же заданных параметрах (ntrees=800)  

In [ ]:
print(rf1.auc(valid=True))

Вся информация идет сначала об обучающей, потом о валидационной выборке
* Отображаются все метрики (MSE, RMSE, LogLoss, AUC, Gini и т.д.)
* Строится матрица ошибок (confusion matrix), которая приводится для порогового значения спрогнозированной вероятности события, оптимального с точки зрения F1-меры _(для справки : F1- мера = 2 x точность x полнота/(точность + полнота))_
* _Maximum Metrics:_ Рассчитываются на ней различные метрики и соответствующие пороговые значения
* _Gains/Lift Table:_ Таблица выигрышей создается путем разбиения данных на группы по квантильным пороговым значениям спрогнозированной верроятности положительного класса
* _Variable Importances:_ Информация о важностях предикторов. Информация выводится также в отмасштабированном и процентном видах. На сайте с документацией указано, что важность рассчитывается, как относительное влияние каждой переменной: была ли переменная выбрана при построении дерева и на как изменилась среднеквадратичная ошибка (рассчитывается на всех деревьях)

In [ ]:
# можно нарисовать графиком 
import matplotlib.pyplot as plt

plt.rcdefaults()
fig, ax = plt.subplots()
variables = rf1._model_json['output']['variable_importances']['variable']
y_pos = np.arange(len(variables))
scaled_importance = rf1._model_json['output']['variable_importances']['scaled_importance']
ax.barh(y_pos, scaled_importance, align='center', color='green', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(variables)
ax.invert_yaxis()
ax.set_xlabel('Scaled Importance')
ax.set_title('Variable Importance')
plt.show()

___Описание параметров вызова функции___  
1. Параметры, определяющие задачу  
    * __model_id:__ идентификатор модели
    * __training_frame:__ датасет для построения модели
    * __validation_frame:__ датасет для валидации
    * __nfolds:__ количество фолдов для кросс-валидации (по умолчанию 0)
    * __y:__ имена зависимой переменной
    * __x:__ список названий предикторов
    * __seed:__ random state <br><br>
2. Параметры, задающие сложность дерева
    * __ntrees__: количество деревьев
    * __max_depth:__  максимальная глубина дерева
    * __min_rows:__ минимальное количество наблюдений в терминальном листе
 <br><br>
3. Параметры, определяющие формирование подвыборок
    * __mtries:__ количество случайно отбираемых предикторов для разбиения узла. По умолчанию -1: для классификации корень квадратный из р, для регрессии р / 3, где р - количество предикторов
    * __sample_rate:__ какую часть строк отбирать (от 0 до 1). По умолчанию 0.6320000291 
    * __sample_rate_per_class:__ для построения модели из несбалансированного набора данных. Какую часть строк выбирать для каждого дерева (от 0 до 1) 
    * __col_sample_rate_per_tree:__ какую часть столбцов выбирать для каждого дерева (от 0 до 1, по умоланчанию 1)
    * __col_sample_rate_change_per_level:__ задает изменение отбора столбцов для каждого уровня дерева (от 0 до 2, по умолчанию 1), например: (factor = col_sample_rate_change_per_level)
        * level 1: col_sample_rate
        * level 2: col_sample_rate * factor
        * level 3: col_sample_rate * factor^2
        * level 4: col_sample_rate * factor^3
 <br><br>
4. Параметры, определяющие биннинг переменных
    * __nbins__: (Numerical/real/int only) для каждой переменной строит по крайней мере n интервалов и затем рабивает по наилучшей точке расщепления
    * __nbins_top_level:__  (Numerical/real/int only) определяет максимальное количество интервалов n на вершине дерева. При переходе на увроень ниже делит заданное число на 2 до тех пор, пока не дойдет до уровня nbins 
    * __nbins_cats:__ (Categorical/enums only) каждая переменная может быть разбита максимум на n интерваловю. Более высокие значения могут привести к переобучению.
    * __categorical_encoding:__ схема кодировки для категориальных переменных
        * auto: используется схема enum.
        * enum: 1 столбец для каждого категориального признака (как есть)
        * one_hot_explicit: N+1 новых столбцов для каждого признака с N уровнями
        * binary: не более 32 столбцов для признака (используется хеширование)
        * eigen: выполняет one-hot-encoding и оставляет k главных компонент
        * label_encoder: все категории сортируются в лексикографическом порядке и каждому уровню присваивается целое число, начиная с 0 (например, level 0 -> 0, level 1 -> 1,  и т.д.)
        * sort_by_response: все категории сортируются по среднему значению переменной и каждому уровню присваивается значение (например, для уровня с мин средним ответом -> 0, вторым наименьшим -> 1, и т.д.). 
    * __histogram_type:__ тип гистограммы для поиска оптимальных расщепляющих значений
        * AUTO = UniformAdaptive
        * UniformAdaptive: задаются интервалы одинаковой ширины (max-min)/N
        * QuantilesGlobal: интервалы одинаквого размера (в каждом интервале одинаковое количество наблюдений)
        * Random: задает построение Extremely Randomized Trees (XRT). Случайным образом отбирается N-1 точка расщепления и затем выбирается наилучшее разбиение
        * RoundRobin: все типы гистограмм (по одному на каждое дерево) перебираются по кругу 
 <br><br>
5. Параметры остановки
    * __stopping_rounds:__ задает количество шагов в течение которого должно произойти заданное улучшение(stopping_tolerance:) для заданной метрики (stopping_metric) 
    * __stopping_metric:__  метрика для ранней остановки (по умолчанию логлосс для классификации и дисперсия - для регрессии). Возможные значения: deviance, logloss, mse, rmse, mae, rmsle, auc, lift_top_group, misclassification, mean_per_class_error
    * __stopping_tolerance:__ относительное улучшение для остановки обучения (если меньше, то остановка)

  
  
  
Полный список всех параметров с описанием можно найти на сайте с [официальной документацией](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html)

<font color=green>Отдельно стоит отметить, что биннинг переменных в h2o позволяет добиться улучшения качества и скорости построения леса. </font>

__Количественные переменные__
В классической реализации случайного леса для количественного предиктора с k категориями может быть рассмотрено k-1 вариантов разбиения. (Все значения сортируются и средние значения по каждой паре рассматриваются в качестве точек расщепления).  
В h2o для каждой переменной определяются типы и количество интервалов для разбиения, т.е. создается гистограмма (тип : histogram_type), и количество регулируется двумя параметрами:  nbins и nbins_top_level. Перебор этих параметров позволит улучшить качество леса.

__Качественные переменные__
В классической реализации случайного леса для категориального предиктора с k категориями может быть рассмотрено ${2^{k-1}} - 1$ разбиения (всеми возможными способами).  
В h2o для каждой качественной переменной также определюется типы и количество интервалов для разбиения. Количество регулируется : nbins_cats и nbins_top_level (при переходе на уровень ниже значение nbins_top_level уменьшается в 2 раза до тех пор пока значение больше nbins_cats).  
Пример как происходит разбиение на бины:  
Если количество  категорий меньше значения параметра nbins_cats, каждая категория
получает свой бин. Допустим, у нас есть переменная Class. Если у нее
есть уровни A, B, C, D, E, F, G и мы зададим nbins_cats=8, то будут
сформировано 7 бинов: {A}, {B}, {C}, {D}, {E}, {F} и {G}. Каждая категория
получает свой бин. Будет рассмотрено ${2^6}-1=63$ точки расщепления. Если
мы зададим nbins_cats=10, то все равно будут получены те же самые
бины, потому что у нас всего 7 категорий. Если количество категорий
больше значения параметра nbins_cats, категории будут сгруппированы
в бины в лексикографическом порядке. Например, если мы зададим
nbins_cats=2, то будет сформировано 2 бина: {A, B, C, D} и {E, F, G}. У
нас будет одна точка расщепления. A, B, C и D попадут в один и тот же
узел и будут разбиты только на последующем, более нижнем уровне или
вообще не будут разбиты.  
  
Этот параметр очень важен для настройки, при больших значениях параметра мы можем получить дополнительную случайность в построении 

И в завершение этой части сделаем подбор параметров и попытаемся улучшить модель по разным параметрам

In [ ]:
from h2o.grid.grid_search import H2OGridSearch

In [ ]:
rf_params = {'max_depth': [10, 14, 18, 24]}

rf_grid = H2OGridSearch(model=H2ORandomForestEstimator(ntrees=800, seed=152),
                          grid_id='rf_grid_max_depth',
                          hyper_params=rf_params)
rf_grid.train(X, y, training_frame=training, validation_frame=validation)

# модели, отсортированные по AUC
rf_gridperf = rf_grid.get_grid(sort_by='auc', decreasing=True)
print(rf_gridperf)

# выберем лучшую модель и выведем AUC на тесте 
best_rf = rf_gridperf.models[0]
print(best_rf.auc(valid=True))
print(rf_gridperf.get_hyperparams(0))

In [ ]:
rf_params1 = {'mtries': [3, 5, 7, 9]}

rf_grid1 = H2OGridSearch(model=H2ORandomForestEstimator(ntrees=800, seed=152, max_depth=14),
                          grid_id='rf_grid_mtries1',
                          hyper_params=rf_params1)
rf_grid1.train(X, y, training_frame=training, validation_frame=validation)

# модели, отсортированные по AUC
rf_gridperf1 = rf_grid1.get_grid(sort_by='auc', decreasing=True)
print(rf_gridperf1)

# выберем лучшую модель и выведем AUC на тесте 
best_rf1 = rf_gridperf1.models[0]
print(best_rf1.auc(valid=True))
print(rf_gridperf1.get_hyperparams(0))

теперь к настройке более специфических параметров

In [ ]:
rf_params2 = {'histogram_type': ['UniformAdaptive', 'Random', 'QuantilesGlobal', 'RoundRobin']}

rf_grid2 = H2OGridSearch(model=H2ORandomForestEstimator(ntrees=800, seed=152, max_depth=14, mtries=7),
                          grid_id='rf_grid_hist_type',
                          hyper_params=rf_params2)
rf_grid2.train(X, y, training_frame=training, validation_frame=validation)

# модели, отсортированные по AUC
rf_gridperf2 = rf_grid2.get_grid(sort_by='auc', decreasing=True)
print(rf_gridperf2)

# выберем лучшую модель и выведем AUC на тесте 
best_rf2 = rf_gridperf2.models[0]
print('AUC for the best model: ', best_rf2.auc(valid=True))
print(rf_gridperf2.get_hyperparams(0))

По типу гистограммы RoundRobin оказался самым оптимальным

In [ ]:
rf_params3 = {'col_sample_rate_per_tree': [0.5, 0.6, 0.7, 0.8, 0.9], 
             'sample_rate': [0.5, 0.6, 0.7, 0.8, 0.9] }

rf_grid3 = H2OGridSearch(model=H2ORandomForestEstimator(ntrees=800, seed=152, max_depth=14, mtries=7,
                                                       histogram_type='RoundRobin'),
                          grid_id='rf_grid3',
                          hyper_params=rf_params3)
rf_grid3.train(X, y, training_frame=training, validation_frame=validation)

# модели, отсортированные по AUC
rf_gridperf3 = rf_grid3.get_grid(sort_by='auc', decreasing=True)
print(rf_gridperf3)

# выберем лучшую модель и выведем AUC на тесте 
best_rf3 = rf_gridperf3.models[0]
print('AUC for the best model: ', best_rf3.auc(valid=True))
print(rf_gridperf3.get_hyperparams(0))

### Logistic Regression

#### sklearn LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression().fit(X_train, y_train)
print('AUC для sklearn LogisticRegression: {:.4f}'.format(roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1])))

проверим на отмасштабированных данных

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression().fit(X_train_scaled, y_train)
print('AUC для sklearn LogisticRegression Scaled: {:.4f}'.format(
    roc_auc_score(y_test, logreg.predict_proba(X_test_scaled)[:, 1])))

результаты хуже, чем в лесу  
займемся подбором параметров в H2O

#### H2OGeneralizedLinearEstimator

In [ ]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [ ]:
# создаем экземпляр класса H2OGeneralizedLinearEstimator
glm_model = H2OGeneralizedLinearEstimator(family= "binomial", seed=1000000)
# обучаем модель
glm_model.train(X, y, training_frame= training, validation_frame=validation)

In [ ]:
glm_model.model_performance().show()

In [ ]:
str(glm_model.model_performance().auc())

___Описание параметров вызова функции___  
1. Параметры, определяющие задачу  
    * __model_id:__ идентификатор 
    * __training_frame:__ датасет для построения модели
    * __validation_frame:__ датасет для валидации
    * __nfolds:__ количество фолдов для кросс-валидации (по умолчанию 0)
    * __y:__ имена зависимой переменной
    * __x:__ список названий предикторов
    * __seed:__ random state 
    * __family:__ тип модели (gaussian, binomial, multinomial, ordinal, quasibinomial, poisson, gamma, tweedie   
    * __solver:__ 
        * IRLSM: Iteratively Reweighted Least Squares Method - используется с небольшим количеством предикторов и для l1-регулярзации
        * L_BFGS: Limited-memory Broyden-Fletcher-Goldfarb-Shanno algorithm - используется для данных в большим числом колонок
        * COORDINATE_DESCENT, COORDINATE_DESCENT_NAIVE - экспериментальные
        * AUTO: Sets the solver based on given data and parameters (default)
        * GRADIENT_DESCENT_LH, GRADIENT_DESCENT_SQERR: используется только для family=Ordinal<br><br>
2. Параметры, определяющие регуляризацию   
для справки формула ElasticNet, объединяющая $L_1$ и $L_2$ регуляризацию
$$\large \begin{array}{rcl}
L &=& -\mathcal{L} + \lambda R\left(\textbf W\right) \\
&=& -\mathcal{L} + \lambda \left(\alpha \sum_{k=1}^K\sum_{i=1}^M w_{ki}^2 + \left(1 - \alpha\right) \sum_{k=1}^K\sum_{i=1}^M \left|w_{ki}\right| \right)
\end{array}$$ где $\alpha \in \left[0, 1\right]$

    * __alpha:__  распределение между $L_1$ и $L_2$ регуляризацией. (1 - $L_1$, 0 - $L_2$)
    * __lambda:__ сила регуляризации
    * __lambda_search:__ True / False. Определяет стоит ли начинать поиск $\lambda$, начиная с максимального значения
    * __lambda_min_ratio:__ минимальное значение $\lambda$, используемое при поиске $\lambda$
    * __nlambdas:__ количество шагов при поиске $\lambda$ (по умолчанию 100)
 <br><br>
3. Параметры, влияющие на предобработку предикторов
    * __standardize:__ использовать ли масштабирование
    * __missing_values_handling:__ как работать с пропцщенными значениями (пропускать или испутировать средним)
    * __remove_collinear_columns:__ удалять ли автоматически коллинеарные столбцы при построении модели 
    * __interactions:__ список колонок, из которых буду составлены все возможные пары и использованы для построения модели 
    * __interaction_pairs:__ список уже готовых пар для модели 
 <br><br>

Полный список всех параметров с описанием можно найти на сайте с [официальной документацией](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/glm.html)
  
  
Воспольуземся перебором параметров [H2OGridSearch] для поиска лучшей модели

In [ ]:
hyper_parameters = {'alpha': np.arange(0, 1.05, 0.05).tolist()}

gridsearch = H2OGridSearch(H2OGeneralizedLinearEstimator(family='binomial', lambda_search=True, standardize=True),
                           grid_id="gridresults", hyper_params=hyper_parameters)
gridsearch.train(X, y, training_frame= training, validation_frame=validation)

gridperf = gridsearch.get_grid(sort_by="auc", decreasing=True)
best_model = gridperf.models[0]
print(gridperf)

print('AUC for the best model: ', best_model.auc(valid=True))
print(best_model.summary()['regularization'])

H2O позволяет настроить силу регуляризации.  
Для получения модели с более выоским AUC - надо попробовать предварительно разные способы масштабирования  
И попробовать настроить еще один интересный параметр  : interactions

### Gradient Boosting

#### sklearn GradientBoostingClassifier


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
grb = GradientBoostingClassifier().fit(X_train, y_train)
print('AUC для sklearn GradientBoostingClassifier: {:.4f}'.format(
    roc_auc_score(y_test, grb.predict_proba(X_test)[:, 1])))

#### Gradient Boosting Machine

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
gbm = H2OGradientBoostingEstimator(ntrees = 800, learn_rate = 0.1,seed = 1234)
gbm.train(X, y, training_frame= training, validation_frame=validation)
gbm.model_performance().show()

In [ ]:
print('AUC на валидационной выборке: ',gbm.auc(valid=True))

___Описание параметров вызова функции___  
только специфические, отличаюшиеся от RandomForest
1. Параметры для GBM
    * __learn_rate:__ скорость обучения (от 0 до 1)
    * __learn_rate_annealing:__ уменьшает скорость обучения после построения каждого дерева
    
Для настройки параметров бустинга есть следующие советы от экспертов (Марк Лэндри, Дмитрий Ларько и [github](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.ipynb)): 
* заификисровать количество деревьев константой, подобрать leraning rate (например, learn_rate=0.02, learn_rate_annealing=0.995) 
* в самом конце можно снова вернуться к настройке количества деревьев
* потом необходимо настроить глубину деревьев (max_depth) (чаще всего это 4-10)
* попробовать изменить тип гистрограммы, nbins/nbins_cat
* изменить настройки, определяющие подвыборки (sample_rate, col_sample_rate), чаще всего это 70-80%
* для несбалансированных наборов необходимо настроить параметры, отвечающие за баланс классов (sample_rate_per_class)
* определить критерии остановки 

начинаем настройку

In [ ]:
gbm_params = {'learn_rate': [0.0001, 0.001, 0.01, 0.1, 1]}

gbm_grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees = 800, seed = 1234),
                          grid_id='gbm_grid_learn_rate',
                          hyper_params=gbm_params)
gbm_grid.train(X, y, training_frame=training, validation_frame=validation)

# модели, отсортированные по AUC
gbm_gridperf = gbm_grid.get_grid(sort_by='auc', decreasing=True)
print(gbm_gridperf)

# выберем лучшую модель и выведем AUC на тесте 
best_gbm = gbm_gridperf.models[0]
print(best_gbm.auc(valid=True))
print(gbm_gridperf.get_hyperparams(0))

In [ ]:
gbm_params1 = {'max_depth': [4, 6, 8, 10]}

gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees = 800, seed = 1234, learn_rate=0.001),
                          grid_id='gbm_grid_max_depth',
                          hyper_params=gbm_params1)
gbm_grid1.train(X, y, training_frame=training, validation_frame=validation)

# модели, отсортированные по AUC
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='auc', decreasing=True)
print(gbm_gridperf1)

# выберем лучшую модель и выведем AUC на тесте 
best_gbm1 = gbm_gridperf1.models[0]
print(best_gbm1.auc(valid=True))
print(gbm_gridperf1.get_hyperparams(0))

In [ ]:
gbm_params2 = {'nbins': [2, 8, 16, 32],
               'histogram_type': ['UniformAdaptive', 'Random', 'QuantilesGlobal', 'RoundRobin']}

gbm_grid2 = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees = 800, seed = 1234, 
                                                             max_depth=6, learn_rate=0.001),
                          grid_id='gbm_grid_hist',
                          hyper_params=gbm_params2)
gbm_grid2.train(X, y, training_frame=training, validation_frame=validation)

# модели, отсортированные по AUC
gbm_gridperf2 = gbm_grid2.get_grid(sort_by='auc', decreasing=True)
print(gbm_gridperf2)

# выберем лучшую модель и выведем AUC на тесте 
best_gbm2 = gbm_gridperf2.models[0]
print(best_gbm2.auc(valid=True))
print(gbm_gridperf2.get_hyperparams(0))

В итоге AUC = 0.9450 на валидационной выборке получился немного выше для GradientBoostingMachine, чем для RandomForestClassifier AUC = 0.941686

#### XGBoost

Еще одним из алгоритмов, доступных в H20, из семейства Gradient Boosting Machine является XGBoost. Эта библиотека - очень популярный инструмент для решения Kaggle-задач. Он позволяет строить деервья параллельно, что позволяет решать проблемы скорости обучения. 

__Особенности модели__

Реализация модели поддерживает особенности реализации scikit-learn и R с новыми дополнениями, такими как регуляризация. Поддерживаются три основные формы повышения градиента:

* Алгоритм Gradient Boosting также называется градиентной машиной повышения, включая скорость обучения.
* Stochastic Gradient Boosting с суб-выборкой в строке, столбце и столбце на каждый уровень разделения.
* Регулярное усиление градиента с регуляцией L1 и L2.

__Системные функции__

Библиотека предоставляет систему для использования в различных вычислительных средах, не в последнюю очередь:

* Параллелизация построения дерева с использованием всех ваших ядер процессора во время обучения.
* Распределенные вычисления для обучения очень крупных моделей с использованием кластера машин.
* Внекорпоративные вычисления для очень больших наборов данных, которые не вписываются в память.
* Кэш Оптимизация структуры данных и алгоритма для наилучшего использования аппаратного обеспечения.

__Особенности алгоритма__

Реализация алгоритма была разработана для эффективности вычислительных ресурсов времени и памяти. Цель проекта заключалась в том, чтобы наилучшим образом использовать имеющиеся ресурсы для обучения модели. Некоторые ключевые функции реализации алгоритма включают:

* Редкая реализация Aware с автоматической обработкой отсутствующих значений данных.
* Блочная структура для поддержки распараллеливания конструкции дерева.
* Продолжение обучения, чтобы вы могли еще больше повысить уже установленную модель для новых данных. 

В реальности в H2O используется нативный алгоритм [XGBoost](http://xgboost.readthedocs.io/en/latest/get_started/index.html), поэтому не буду пытатьтся подобрать параметры, покажу только, как запускается

In [ ]:
from h2o.estimators import H2OXGBoostEstimator

# библиотека доступна не для всех платформ, сначала надо проверить ее доступность
is_xgboost_available = H2OXGBoostEstimator.available()
print(is_xgboost_available)

In [ ]:
param = {
      "ntrees" : 100
    , "max_depth" : 10
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
}

model = H2OXGBoostEstimator(**param)
model.train(X, y, training_frame=training, validation_frame=validation)
model.model_performance().show()

print('AUC на валидационной выборке GradientBoostingMachine: ',gbm.auc(valid=True))
print('AUC на валидационной выборке XGBoost: ',model.auc(valid=True))

#### LightGBM

[LightGBM](https://github.com/Microsoft/LightGBM) строит глубокие асимметричные деревья, повторно разбивая один лист вместо разбиения всех листьев одного уровня до достижения максимальной глубины. 
XGBoost использует предварительную сортировку и гистограммирование для расчета наилучшего разбиения, т.е.
- для каждого узла необходимо пронумеровать все признаки; 
- для каждого признака необходимо провести сортировку всех значений (здесь можно разбить на бины и провести сортировку бинов); 
- ищем налиучшее разбиение для признака; 
- выбираем наилучшее разбиение среди всех признаков.

но LightGBM использует другой способ: градиент - это угол наклона функции потерь, таким образом, если градиент для каких-то точек больше, то эти точки важнее для поиска оптимального разбиения. Алгоритм находит все такие точки с максимальным градиентом и делает рандомное расщепление на точках с маленький градиентом.  
Предположим, есть 500K строчек данных, где у 10k строчек градиент больше, таким образом алогритм выберет 10k строчку большим градиентом + x% от отсавшихся 490k строчек, выбранных случайно. Предположим, x = 10%, общее количество выбранных строк = 59k из 500K
Важное предположение здесь: ошибка на тренировочном наборе с меньшим градиентом меньше и эти данные уже хорошо обучены в модели. Таким образом мы уменьшаем с одной стороны количество данных для обучения, но при этом сохраняем качество для уже обученных деревьев.

----------------
H2O не интегрирован с LightGBM, но предоставляет метод для эмуляции LightGBM алгоритма, используя определенный набор параметров:


tree_method="hist"
grow_policy="lossguide"

In [ ]:
h2o.cluster().shutdown()

__Список литературы__

* https://www.h2o.ai/  
* http://statistica.ru/local-portals/actuaries/obobshchennye-lineynye-modeli-glm/  
* https://en.wikipedia.org/wiki/Generalized_linear_model  
* https://alexanderdyakonov.files.wordpress.com/2017/06/book_boosting_pdf.pdf  
* https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.ipynb  
* https://ru.bmstu.wiki/XGBoost  
* Артем Груздев. Прогнозное моделирование в IBM SPSS Statistics, R и Python; Лекции
* https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db